<a href="https://colab.research.google.com/github/pbenito1/TFM/blob/main/CZDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción contra dominios `.COM` del servicio CZDS

El servicio CZDS de [ICANN](https://czds.icann.org/home) proporciona el listado de los dominios de una gran parte de los dominios TLD disponibles.

Entre ellos, se encuentra el TLD más popular, el "_.com_" que cuenta en la actualidad con casi 160 millones de dominios, y del que se crean diariamente [más de 100.000 dominios](https://dnpedia.com/tlds/daily.php).

En este experimento vamos a realizar la predicción de bondad del listado de dominios .com de acuerdo a los modelos entrenados en las fases anteriores del TFM.

## Pretratamiento de dominios `.com``

El listado de dominios `.com` proporcionado por ICANN se proporciona en forma de un archivo denominado `com.txt.gz` que contiene un único archivo `com.txt` con el siguiente contenido:

```
com.    900     in      soa     a.gtld-servers.net.     nstld.verisign-grs.com. 1664841622      1800    900     604800  86400
0-------------------------------------------------------------0.com.    172800  in      ns      ns1.domainit.com.
0-------------------------------------------------------------0.com.    172800  in      ns      ns2.domainit.com.
0-------------------------------------------------------------5.com.    172800  in      ns      fns.frogsmart.net.
0-------------------------------------------------------------5.com.    172800  in      ns      sns.frogsmart.net.
0-------------------------------------------------------------5.com.    172800  in      ns      tns.frogsmart.net.
0---------------------0.com.    172800  in      ns      ns0.directnic.com.
0---------------------0.com.    172800  in      ns      ns1.directnic.com.
```

En el mismo se puede observar que contiene no únicamente los dominios registrados, sino los servidores de nombre asociados. Realmente de esa información, en nuestro caso, únicamente nos interesa el nombre de dominio. Por este motivo es necesario realizar un pretratamiento para filtrar, exclusivamente, los dominios `.com` únicos.

Este tratamiento lo realizamos mediante los siguientes comandos en `bash` de Linux:

Descomprimimos el archivo, nos quedamos con la primera columna y deduplicamos los dominios utilizando el comando `uniq`:
```
$ gunzip -c com.txt.gz|cut -f1|uniq|more>com_domains.txt
```

Eliminamos los dominios `unicode`, comienzan por `xn--`:
```
$ grep -v '^xn--' com_domains.txt >com_domains_uniq_noxnn.txt
```

Extraemos el SLD:
```
$ cat com_domains_uniq_noxnn.txt | rev | cut -d'.' -f3 | rev >com_domains_tld.txt
```

Calculamos el número de dominios:
```
$ wc -l com_domains_tld.txt
165107972 com_domains_tld.txt
```

Hay más de 165 millones de dominios `.com`


A continuación cargaremos el modelo CNN previamente entrenado en el _notebook_ de [Redes neuronales](RedesNeuronales.ipynb), y realizaremos las predicciones de parte de los dominios `.com`. Dado que existen limitaciones de memoria en el servicio Google Colab, se realizarán las predicciones únicamente sobre 10 millones de registros de CZDS.


In [1]:
# imports
import os
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

In [3]:
# Cargamos el modelo CNN
GDRIVE_PATH='/content/drive/MyDrive/TFM'
CHECKPOINTS_PATH=os.path.join(GDRIVE_PATH, 'checkpoints')

Cargamos el modelo entrenado previamente

In [4]:
model = tf.keras.models.load_model(os.path.join(GDRIVE_PATH,'CONV1D'))

Cargamos el mapeo de caracteres previamente serializado

In [5]:
with open(os.path.join(GDRIVE_PATH, 'valid_chars.pickle'), 'rb') as h:
  valid_chars=pickle.load(h)

Cargamos el archivo preprocesado con los dominios `.com` de CZDS. 

Por limitaciones de memoria únicamente cargamos 20 millones de elementos. De ellos tomamsos una muestra de 50.000 elementos

In [7]:
df = pd.read_csv(os.path.join(GDRIVE_PATH,'CZDS/com_domains_tld.txt'),header=None,names=['domain'],nrows=20000000)
df=df.sample(50000)

Añadimos el TLD `.com`. La red neuronal toma como parámetro el dominio completo, incluyendo TLD.

In [9]:
df['full_domain'] = df['domain'].apply(lambda d:d+'.com')

In [14]:
X = np.array(df['full_domain'])

Mapeamos los caracteres a su código numérico asociado

In [15]:
X = [[valid_chars[y] for y in x] for x in X]

Añadimos _padding_ hasta la máxima longitud del dominio (67)

In [16]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=67)

In [17]:
p=model.predict(X)

1563/1563 [==============================] - 10s 2ms/step


Las predicciones menores de 0.5 se consideran benignas y las mayores, malignas.

In [18]:
Y = np.array(['benign' if x <0.5 else 'malign' for x in p])

Añadimos los resultados al _dataframe_ original

In [21]:
np.concatenate( p, axis=0 )
df['category'] = Y.tolist()
df['prediction'] = np.concatenate( p, axis=0 )

Muestra de **dominios benignos**. Se toman los valores de predicción más cercanos a 0, los cuales son más representativos:

In [22]:
#df[df.category=='benign'].sample(10)
df[df.prediction<0.01].sample(10)

,domain,full_domain,category,prediction
13665165,ar4it,ar4it.com,benign,0.007594
4966820,778sold,778sold.com,benign,0.006028
10406784,allaroundcard,allaroundcard.com,benign,0.004897
8660566,agence-rhodium,agence-rhodium.com,benign,0.001894
19505962,bellalogic,bellalogic.com,benign,0.000576
3588549,500rule,500rule.com,benign,0.001102
541912,100prozenthandmade,100prozenthandmade.com,benign,0.000346
5634149,8dconsulting,8dconsulting.com,benign,0.007053
17887312,bangwebsites,bangwebsites.com,benign,0.000232
13450699,appmetics,appmetics.com,benign,0.000241


Muestra de **dominios malignos**. Se toman los valores de predicción más cercanos a 1, los cuales son más representativos:

In [23]:
#df[df.category=='malign'].sample(10)
df[df.prediction>0.95].sample(10)

,domain,full_domain,category,prediction
17063519,b2odsh18h2vgr66barnsfgmqpm44bltv,b2odsh18h2vgr66barnsfgmqpm44bltv.com,malign,0.999998
2893668,3mhqc3sts4sida5oq17pd39i3irb4qv2,3mhqc3sts4sida5oq17pd39i3irb4qv2.com,malign,1.000000
4716814,6unl826nc492tb8900inelbse033cbtm,6unl826nc492tb8900inelbse033cbtm.com,malign,0.999987
5677386,8j6jp7140ffre4e48t7km7rl2hvmnrcr,8j6jp7140ffre4e48t7km7rl2hvmnrcr.com,malign,1.000000
5764769,8v9s7ek18emt9f8vd8j4sc8ughr41e74,8v9s7ek18emt9f8vd8j4sc8ughr41e74.com,malign,1.000000
5257562,802g3iik1p1umnpg0mp0as01fuh8grm7,802g3iik1p1umnpg0mp0as01fuh8grm7.com,malign,1.000000
4063884,5e9iaqgndumv921oookcpk7n65bs6l03,5e9iaqgndumv921oookcpk7n65bs6l03.com,malign,1.000000
2718555,3b3lte0sotbmb6755464ai3595mj2trj,3b3lte0sotbmb6755464ai3595mj2trj.com,malign,1.000000
404661,0kf6hqusl8okdvghko3t7ba5clk9549u,0kf6hqusl8okdvghko3t7ba5clk9549u.com,malign,1.000000
4019674,59td4b0qlguojd1g3c8837glhtoa1hu6,59td4b0qlguojd1g3c8837glhtoa1hu6.com,malign,1.000000


Los dominios malignos, aparentemente, tienen apariencia de DGA.

Veamos a continuación cuantos de los dominios se consideran benignos y cuantos malignos:

In [24]:
df['category'].value_counts(ascending=False)

benign    43277
malign     6723
Name: category, dtype: int64